In [1]:
import re
from google.colab import drive
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
def transforma_mes(x):
  if x[0:3].lower() == 'jan':
    return 1
  elif x[0:3].lower() == 'feb':
    return 2
  elif x[0:3].lower() == 'mar':
    return 3
  elif x[0:3].lower() == 'apr':
    return 4
  elif x[0:3].lower() == 'may':
    return 5
  elif x[0:3].lower() == 'jun':
    return 6
  elif x[0:3].lower() == 'jul':
    return 7
  elif x[0:3].lower() == 'aug':
    return 8
  elif x[0:3].lower() == 'sep':
    return 9
  elif x[0:3].lower() == 'oct':
    return 10
  elif x[0:3].lower() == 'nov':
    return 11
  elif x[0:3].lower() == 'dec':
    return 12

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
def dates(archivo):
  try:
    f = open('/content/drive/My Drive/Modulo 9/' + archivo + '.txt', 'r')
    dates = f.read() # Se guarda el texto en la variable texto
    print('Archivo cargado correctamente')
  except: # En caso de ingresar un nombre incorrecto se imprime un mensaje y termina la función
    print('Archivo no encontrado')

  # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
  fechas1 = re.findall(r'(?:\d{2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]? [12]\d{3}', dates, flags = re.I) # Regexp
  fechas1_ = [fecha.split(' ') for fecha in fechas1] # Separación de cada fecha por espacios
  fechas1_ = [[1] + fecha_list  if len(fecha_list) == 2 else fecha_list for fecha_list in fechas1_  ] # Colocamos el primer día de mes a aquellas fechas que no tienen específicado el día
  fechas1_ = pd.DataFrame(fechas1_, columns = ['day', 'month', 'year']) # Creación del dataframe con estas fechas
  fechas1_['month'] = fechas1_['month'].apply(transforma_mes) # Transforrmamos los meses a número

  # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
  fechas2 = re.findall('(\d{1,2}[/-]\d{1,2}[/-]\d{2})\D', dates) # Regexp para fechas con año de dos cifras
  fechas2 = fechas2 + re.findall('(\d{1,2}[/-]\d{1,2}[/-][12]\d{3})', dates) # Regexp para fechas con año de cuatro cifras

  # Creamos dos listas: separando con - y con /
  fechas2A_ = [fecha.split('-') for fecha in fechas2] # Separamos cada fecha por -
  fechas2A_ = pd.DataFrame(fechas2A_, columns = ['month', 'day', 'year']).dropna(axis = 0) # Eliminamos los nulos para quedarnos solo con los que si tienen originalmente -

  fechas2B_ = [fecha.split('/') for fecha in fechas2] # Separamos cada fecha por /
  fechas2B_ = pd.DataFrame(fechas2B_, columns = ['month', 'day', 'year']).dropna(axis = 0) # Eliminamos los nulos para quedarnos solo con los que si tienen originalmente /

  fechas2_ = pd.concat([fechas2A_, fechas2B_]) # Creamos un dataframe con las dos listas anteriores
  fechas2_['year'] = fechas2_['year'].apply(lambda x: int(x) + 1900 if len(str(x)) == 2 else x) # Transformamos aquellas fechas con formato tipo 82 a 1982


  # Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009
  fechas3 = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\D{1,2}\d{1,2}[a-z]{0,2}[.,-/]? [12]\d{3}', dates, flags = re.I) # Regexp

  fechas3_ = [fecha.split(' ') for fecha in fechas3] # Separamos las fechas por espacios
  fechas3_ = pd.DataFrame(fechas3_, columns = ['month', 'day', 'year']) # Creamos dataframe
  fechas3_['month'] = fechas3_['month'].apply(transforma_mes) # Transformamos los meses a número
  fechas3_['day'] = fechas3_['day'].apply(lambda x: x[0:2]) # Eliminamos las comas y puntos que pueden existir

  # 6/2008; 12/2009
  fechas4 = re.findall(r'\D{2,}(\d{1,2}[/-][12]\d{3})', dates, re.I) # Regexp para evitar repetir las fechas de tipo dd/mm/aaaa
  fechas4 = fechas4 + re.findall(r'\d[\n ](\d{1,2}[/-][12]\d{3})', dates) # Agregamos fechas que pueden tener un espacio o salto de línea antes

  fechas4_ = [fecha.split('/') for fecha in fechas4] # Separamos fechas por /
  fechas4_ = pd.DataFrame(fechas4_, columns = ['month', 'year']) # Creamos dataframe
  fechas4_['day'] = 1 # Colocamos el primer día del mes a todas las fechas puesto que no lo tienen específicado

  # 2009, 2010
  fechas5 = re.findall(r'(?:in|quit|since|delivery|nephrectomy|-|:|;)\D{1,2}([12]\d{3})', dates, flags = re.I) # Regexp de años del tipo sine 1900
  fechas5 = fechas5 + re.findall(r'[a-z\.\n]([12]\d{3})', dates, flags = re.I) # fechas de tipo s1900
  fechas5 = fechas5 + re.findall(r'[(]([12]\d{3})[)]', dates, flags = re.I) # Fechas de tipo (1900)
  fechas5 = fechas5 + re.findall(r'([12]\d{3})-[-a-z]', dates, flags = re.I) # Fechas de tipo 1900-- o 1900-new
  fechas5 = fechas5 + re.findall(r'\d{3,} ([12]\d{3})', dates, flags = re.I) # fechas de tipo 123 1900

  fechas5_ = pd.DataFrame(fechas5, columns = ['year']) # Creación de dataframe
  fechas5_['month'] = 1 # Agregamos un 1 al mes
  fechas5_['day'] = 1 # Agregamos un 1 al día 


  # Lista de fechas final
  fechas_ = pd.concat([fechas1_, fechas2_, fechas3_, fechas4_, fechas5_]) # Concatenamos todos los dataframes creados
  fechas_['fecha'] = fechas_.apply(lambda x: str(x['day']) + '/' + str(x['month']) + '/' + str(x['year']), axis = 1) #  Creamos una nueva columna juntando dia mes y año en formato d/m/a

  fechas_['fecha'] = pd.to_datetime(fechas_['fecha'], dayfirst = True) # Transformamos a formato de fecha la columna fecha


  fechas_ = fechas_.sort_values('fecha') # Ordenamos el dataframe de acuerdo a la columna fecha

  fechas_list = [str(fecha) for fecha in fechas_['fecha'].dt.date] # Creamos una lista con la columna fecha del dataframe
  
  print(f"Se tiene un total de {len(fechas_list)} fechas")
  print(f"Lista de fechas ordenadas: {fechas_list}")
  
  return fechas_list # Retornamos la lista

In [37]:
fechas = dates('dates')

Archivo cargado correctamente
Se tiene un total de 500 fechas
Lista de fechas ordenadas: ['1971-04-10', '1971-05-18', '1971-07-08', '1971-07-11', '1971-09-12', '1972-01-01', '1972-01-13', '1972-01-26', '1972-05-06', '1972-05-13', '1972-06-10', '1972-06-15', '1972-07-20', '1972-10-04', '1972-11-30', '1973-01-01', '1973-02-01', '1973-02-01', '1973-02-14', '1973-03-01', '1973-03-01', '1973-04-01', '1973-06-01', '1973-07-01', '1973-10-01', '1973-12-01', '1974-01-01', '1974-02-01', '1974-02-24', '1974-03-01', '1974-03-05', '1974-03-06', '1974-04-01', '1974-04-11', '1974-04-12', '1974-05-04', '1974-05-26', '1974-06-01', '1974-06-13', '1974-07-07', '1974-08-01', '1974-09-01', '1974-09-10', '1974-10-10', '1974-10-14', '1975-01-01', '1975-02-28', '1975-03-01', '1975-04-09', '1975-07-01', '1975-07-29', '1975-08-01', '1975-08-18', '1975-09-01', '1975-09-01', '1975-09-27', '1975-11-22', '1975-12-01', '1975-12-14', '1976-02-01', '1976-02-11', '1976-03-01', '1976-03-10', '1976-04-01', '1976-06-01', 